# Import package

In [ ]:
import os
import math
import time 
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
from collections import Counter

# Global variable

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(device_lib.list_local_devices())
label_classes = ['森森', '牛排', '巧巧', '小白', '花捲', '金毛', '吊吊', '黑胖', '乖狗', '橘子']

num_of_class = len(label_classes)
batch_size = 32
target_size = (224, 224)

# Preprocess data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# train
train_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)
for i, j in Counter(train_generator.classes).items():
    print(f"{label_classes[i]} with {j:3d} images")
print()

# valid
valid_generator = train_datagen.flow_from_directory(
    directory="data/",
    target_size=target_size,
    color_mode="rgb",
    class_mode='categorical',
    classes=label_classes,
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
)
for i, j in Counter(valid_generator.classes).items():
    print(f"{label_classes[i]} with {j:3d} images")

In [ ]:
for my_batch in train_generator:
    images = my_batch[0]
    labels = my_batch[1]
    for i in range(3):
        print(label_classes[labels[i].argmax()])
        plt.imshow(images[i])
        plt.show()
    break

# Build model

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(
    include_top = False,
    input_shape = target_size + (3, ),
    weights = 'imagenet'
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Conv2D(128, 3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_of_class, activation='softmax')
])

model.summary()

# Start training

In [ ]:
my_callback = [tf.keras.callbacks.ModelCheckpoint(
    filepath='model_save/model_{epoch:03d}_{val_accuracy:.2f}.hdf5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)]

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=10,
    callbacks=my_callback
)

acc = history.history['accuracy']
epochs = range(1, len(acc) + 1)
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc', linewidth=0.5)
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.grid()
# 儲存acc學習曲線
# plt.savefig('./acc.png')
plt.show()

# 畫出loss學習曲線
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss', linewidth=0.5)
plt.title('Training and validation loss')
plt.legend(loc='upper right')
plt.grid()
# 儲存loss學習曲線
# plt.savefig('loss.png')
plt.show()

In [ ]:
pred_train = model.evaluate(train_generator, verbose=0)
pred_val = model.evaluate(valid_generator, verbose=0)

print("Train loss:", pred_train[0])
print("Train accuracy:", pred_train[1])
print("Validation loss:", pred_val[0])
print("Validation accuracy:", pred_val[1])

In [ ]:
# prediction = model.predict(valid_generator, verbose=0)
# prediction = np.argmax(prediction, axis=1)

# images = valid_generator[0][0]
# labels = np.argmax(valid_generator[0][1], axis=1)

# for i in range(9):
#     print(f"label={label_classes[labels[i]]} | predict={label_classes[prediction[i]]}")
#     plt.imshow(images[i])
#     plt.show()

# Convert to tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model.tflite", "wb") as fh:
    fh.write(tflite_model)